In [1835]:
from eth_utils import address
from web3 import Web3
import os
import solcx
from solcx import compile_standard, install_solc
from dotenv import load_dotenv
import json
import numpy as np
import random as rd
import hashlib 
from phe import paillier
import random as rd
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import rsa
from random import choice


In [1836]:
listOfUID = [1,2]
listofBID = []
dictionaryOfPublicKeys = {}
dictionaryOfPrivateKeys = {}
dictOfRSAPubKey,dictOfRSAPriKey = {},{}
publicKeyAuc, privateKeyAuc,RSApublicKeyAuc, RSAprivateKeyAuc = 0,0,0,0
bidderHighest,bidHighest  = 0 , 0
k = 0

In [1837]:
# opening the SCR file

with open("./SCR.sol", "r") as file:
    registrationFile = file.read()

# Opening the SCA File
    
with open("./SCA.sol", "r") as file:
    AuctioneerFile = file.read()

In [1838]:
#compiling smart contract of certifier
install_solc("0.8.2")
compiled_sol = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCR.sol": {"content": registrationFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_sol, file)

In [1839]:
compiled_sol

{'contracts': {'SCR.sol': {'Registration': {'abi': [{'inputs': [],
      'name': 'retrieve',
      'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [{'internalType': 'uint256',
        'name': 'UID',
        'type': 'uint256'}],
      'name': 'store',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'}],
    'evm': {'bytecode': {'generatedSources': [],
      'linkReferences': {},
      'object': '608060405234801561001057600080fd5b5061012f806100206000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c80632e64cec11460375780636057361d146051575b600080fd5b603d6069565b6040516048919060c2565b60405180910390f35b6067600480360381019060639190608f565b6072565b005b60008054905090565b8060008190555050565b60008135905060898160e5565b92915050565b60006020828403121560a057600080fd5b600060ac84828501607c565b91505092915050565b60bc8160db565b82525050565b600060208201

In [1840]:
# get bytecode
bytecode = compiled_sol["contracts"]["SCR.sol"]["Registration"]["evm"]["bytecode"]["object"]

# get abi
abi = json.loads(
    compiled_sol["contracts"]["SCR.sol"]["Registration"]["metadata"]
)["output"]["abi"]

In [1841]:
abi

[{'inputs': [],
  'name': 'retrieve',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256', 'name': 'UID', 'type': 'uint256'}],
  'name': 'store',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [1842]:
# compiling SmartContract of auctioneer
install_solc("0.8.2")
compiled_solA = compile_standard(
    {
        "language": "Solidity",
        "sources": {"SCA.sol": {"content": AuctioneerFile}},
        "settings": {
            "outputSelection": {
                "*": {
                    "*": ["abi", "metadata", "evm.bytecode", "evm.bytecode.sourceMap"]
                }
            }
        },
    },
    solc_version="0.8.2",
)

with open("compiled_code.json", "w") as file:
    json.dump(compiled_solA, file)

In [1843]:
compiled_solA

{'contracts': {'SCA.sol': {'Submission': {'abi': [{'inputs': [],
      'name': 'AuctioneerRetrievesBidder2EncryptedData',
      'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [],
      'name': 'AuctioneerRetrivesBidder1EncryptedData',
      'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
      'stateMutability': 'view',
      'type': 'function'},
     {'inputs': [{'internalType': 'uint256',
        'name': 'num',
        'type': 'uint256'}],
      'name': 'bidderEncryptedBidSubmission',
      'outputs': [],
      'stateMutability': 'nonpayable',
      'type': 'function'}],
    'evm': {'bytecode': {'generatedSources': [],
      'linkReferences': {},
      'object': '608060405234801561001057600080fd5b5061017a806100206000396000f3fe608060405234801561001057600080fd5b50600436106100415760003560e01c80630fbc3b8a14610046578063337b926714610062578063ee643f6814610080575b600

In [1844]:
# get bytecode
bytecodeA = compiled_solA["contracts"]["SCA.sol"]["Submission"]["evm"]["bytecode"]["object"]

# get abi
abiA = json.loads(
    compiled_solA["contracts"]["SCA.sol"]["Submission"]["metadata"]
)["output"]["abi"]

In [1845]:
abiA

[{'inputs': [],
  'name': 'AuctioneerRetrievesBidder2EncryptedData',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'AuctioneerRetrivesBidder1EncryptedData',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'uint256', 'name': 'num', 'type': 'uint256'}],
  'name': 'bidderEncryptedBidSubmission',
  'outputs': [],
  'stateMutability': 'nonpayable',
  'type': 'function'}]

In [1846]:
# set up connection for certifier
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_address = "0x19cfa72e6e0a5aF136111EAC2A96ed744b9b9627"
#private_key = os.getenv("PRIVATE_KEY")
private_key = "0xd54a7c95acc41adb4c5538dfcf8f5b7304f10ce94eb35d29dec7f801a99a74cd"

In [1847]:
# set up connection for auctioneer
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))
chain_id = 1337
my_addressA = "0x5CB5c633033371f642161Ee73147a4Bf4AD07493"
#private_key = os.getenv("PRIVATE_KEY")
private_keyA = "0xfc5f8f7fca9db1d890858548ec026c96f189501d5a1b1ceeb92e74b7b4f38670"

In [1848]:
def registrationPhaseFunctiontoInitializeSCR():
    # initialize contract
    Registration = w3.eth.contract(abi=abi, bytecode=bytecode)
    nonce = w3.eth.get_transaction_count(my_address)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction = Registration.constructor().build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_key)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    return [tx_receipt, nonce]

In [1849]:
def registrationFunctionForSCRToStoreAndRetrieveUID (tx_receipt, nonce,UID):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    registration_sol = w3.eth.contract(abi=abi, address=tx_receipt.contractAddress)

    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction

    call_fun = registration_sol.functions.store(UID).build_transaction(
        {"chainId": chain_id, "from": my_address, "nonce": nonce + 1}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_key)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

    # just viewing something from your transaction doesnt cost anyting

    print(registration_sol.functions.retrieve().call())
    return registration_sol.functions.retrieve().call()


In [1850]:
def bidSubmissionPhaseFunctiontoInitializeSCA():
    # initialize contract
    bidSubmission = w3.eth.contract(abi=abiA, bytecode=bytecodeA)
    nonce = w3.eth.get_transaction_count(my_addressA)
    # set up transaction from constructor which executes when firstly we are trasacting the smart contract

    transaction =bidSubmission.constructor().build_transaction(
        {"chainId": chain_id, "from": my_addressA, "nonce": nonce}
    )
    # Second requirement sign the transaction
    signed_tx = w3.eth.account.sign_transaction(transaction, private_key=private_keyA)

    # Third requirement is to get the transation hash value

    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

    #  after geeting hash get the recipetpt of the transaction
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

    return [tx_receipt, nonce]

In [1851]:
def bidSubmissionFunctionForSCAToStoreAndRetrievePB (tx_receipt, nonce,pb):
    # calling functions in contract block
    # to work with a contract, you need abi and address
    #  this line call the the contract by passing the abi and adress of the transactoin

    bid_submission_sol = w3.eth.contract(abi=abiA, address=tx_receipt.contractAddress)

    #  whenever we want to uipdate something we trsact that value so gas is used for that purpose we agin perfomr the transaction

    call_fun = bid_submission_sol.functions.bidderEncryptedBidSubmission(pb).build_transaction(
        {"chainId": chain_id, "from": my_addressA, "nonce": nonce + 1}
    )
    # again sign and same procedure

    sign_call_fun = w3.eth.account.sign_transaction(call_fun, private_key=private_keyA)
    tx_call_fun_hash = w3.eth.send_raw_transaction(sign_call_fun.rawTransaction)
    tx_call_fun_receipt = w3.eth.wait_for_transaction_receipt(tx_call_fun_hash)

    # just viewing something from your transaction doesnt cost anyting

    print(bid_submission_sol.functions.retrieve().call())
    return bid_submission_sol.functions.retrieve().call()


FROM this line the code for origiianl setup works


this phase consist of the registration phase

In [1852]:
def bidderAndAuctioneerRegistration():
    global RSAprivateKeyAuc, RSApublicKeyAuc, privateKeyAuc,publicKeyAuc
    publicKeyAuc, privateKeyAuc = paillier.generate_paillier_keypair(n_length= 64)
    RSApublicKeyAuc, RSAprivateKeyAuc = rsa.newkeys(1048)
    for i in range(len(listOfUID)):
        #call the smart contract here for registration
        values = registrationPhaseFunctiontoInitializeSCR()
        UID = registrationFunctionForSCRToStoreAndRetrieveUID(values[0],values[1],i)
        listofBID.append(hashlib.sha256(str(UID).encode('utf-8')).hexdigest())
        x,y = paillier.generate_paillier_keypair(n_length=64)
        dictionaryOfPublicKeys[listofBID[i]],dictionaryOfPrivateKeys[listofBID[i]]  = x ,y
        (pubkey, privkey) = rsa.newkeys(1048)
        dictOfRSAPubKey[i],dictOfRSAPriKey[i] = pubkey, privkey 
    

        

this phase consists of the bid submission phase

In [1853]:
def generateTwoBiddersSTPCData(flag,bidderHighest):
    #add bid higest of hihgest bidder later
    bidder1 = 0
    if flag == 0:
        temp = rd.randint(0,len(listOfUID))
        bidder1 = listofBID[temp]
        b1rdbid = rd.randint(1,200)
    else: 
        bidder1 = bidderHighest
        b1rdbid = rd.randint(1,200)
    bidder2 = choice([i for i in listofBID if i not in [bidderHighest]])
    b1rdnum = rd.randint(1,2000) 
    b2rdnum = rd.randint(1,2000)
    b2rdbid = rd.randint(1,200)
    return bidder1,bidder2,b1rdnum,b1rdbid,b2rdnum,b2rdbid

In [1854]:
def bidSubmission(flag,bidderHighest):
    global k
    bidder1,bidder2,b1rdnum,b1rdbid,b2rdnum,b2rdbid = generateTwoBiddersSTPCData(flag,bidderHighest)

    # Generate the pb12 and send to smart contract
    pb12 = rsa.encrypt(str((dictionaryOfPublicKeys[bidder2].raw_encrypt(b1rdbid + b1rdnum))).encode('utf8'), RSApublicKeyAuc)
    # print(type(pb12))
    # print(pb12)
    k = Web3.to_int(pb12) 
    print(type(k))
    values = bidSubmissionPhaseFunctiontoInitializeSCA()
    pb12 = bidSubmissionFunctionForSCAToStoreAndRetrievePB(values[0],values[1],k)
    

    # Generate the pb21 and send to smart contract
    pb21 = rsa.encrypt(str((dictionaryOfPublicKeys[bidder1].raw_encrypt(b2rdnum+ b2rdbid))).encode("utf8"), RSApublicKeyAuc)
    pb21 = Web3.to_hex()
    values = bidSubmissionPhaseFunctiontoInitializeSCA()
    pb21 = bidSubmissionFunctionForSCAToStoreAndRetrievePB(values[0],values[1],pb21)
    
    #call smartcontract SCA frrom here to submit these values to the auctioneer
    pb12 = int(rsa.decrypt(pb12,RSAprivateKeyAuc).decode('utf8'))
    pb21 = int(rsa.decrypt(pb21,RSAprivateKeyAuc).decode('utf8'))
    #auctioneer adds its random number fraction here
    m1 = rd.randint(1,10000)
    m2 = rd.randint(1,10000)
    pb12 = np.dot(pb12, dictionaryOfPublicKeys[bidder2].raw_encrypt(m1+m2))
    pb21 = np.dot(pb21, dictionaryOfPublicKeys[bidder1].raw_encrypt(m1+m2))
    # auctioneer forwards this info to b1 and b2 and they both add their random numbers
    #data need to be submited here to smart contract
    pb21 = dictionaryOfPrivateKeys[bidder1].raw_decrypt(pb21)
    pb12 = dictionaryOfPrivateKeys[bidder2].raw_decrypt(pb12)
    return [pb12,pb21,bidder1,bidder2]

THIS ENDS THE BID SUBSMISSION AND BID COMPARISON STARTS FROM HERE


In [1855]:
#add bids comarison pairs in parmeters
def bidComparison(pb12,pb21,bidder1,bidder2):
    if pb12 < pb21:
        return bidder2
    elif pb12 > pb21:
        return bidder1
    else:
        #0 means a tie
        return 0

this part of the code simulates the envioronment where bidders want to raise the bid

In [1856]:
# SCA smart cotract reopens here
def bidRaise():
    global bidderHighest
    x = bidSubmission(1,bidderHighest)
    bidderHighest = bidComparison(x[0],x[1],x[2],x[3])
    print(bidderHighest)

In [1857]:
# intitializing the first bid value for two random bidders
def firstBidding():
    global bidderHighest
    x = bidSubmission(0,0)
    bidderHighest = bidComparison(x[0],x[1],x[2],x[3])
    print(bidderHighest)

In [1858]:
bidderAndAuctioneerRegistration()

0
1


In [1859]:

firstBidding()
bidRaise()
# bidRaise()
# bidRaise()
# bidRaise()

<class 'int'>


Web3ValidationError: 
Could not identify the intended function with name `bidderEncryptedBidSubmission`, positional arguments with type(s) `int` and keyword arguments with type(s) `{}`.
Found 1 function(s) with the name `bidderEncryptedBidSubmission`: ['bidderEncryptedBidSubmission(uint256)']
Function invocation failed due to no matching argument types.

In [ ]:
k


56

In [ ]:
k = Web3.to_int(k)

In [ ]:
type(k)

int

In [ ]:
m = Web3.to_bytes(k)

In [ ]:
m

b'8'